In [2]:
# Trained Model
import pandas as pd
import numpy as np
from chainer import serializers
from chainer import Link, Chain, ChainList
import chainer.links as L
import chainer.functions as F
##################################################################################
##################################################################################
#### Hi, Bruce. Frst of all, run the following code (from line 5 to line 49). ####
##################################################################################
##################################################################################
##################################################################################


C:\Users\bruce\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
ticker_df = pd.read_csv('stock_data/ticker_data.csv') ## Please change the directory where you save stock_data file.
factor_df = pd.read_csv('stock_data/factor_data.csv') ## Same as above.
ticker_df.set_index('timestep', inplace=True)
factor_df.set_index('timestep', inplace=True)


In [5]:
average = factor_df.mean()
std = factor_df.std()

factor_df = (factor_df - average)/std
stock_df = ticker_df.join(factor_df, how='left')
subdata = stock_df[stock_df['ticker']==0]

In [7]:
X_training = subdata[['VIX', 'COPP', "3M_R", "US_TRY", "BIG_IX","SMALL_IX", "SENTI", "TEMP", "RAIN", "OIL"]]
X_training = X_training.as_matrix(columns=None)
X_training = np.ndarray.astype(X_training, "float32")

Y = subdata['returns']
Y_training = pd.to_numeric(round(Y*2000+30),downcast='unsigned')
Y_training = Y_training.as_matrix(columns=None)
Y_training = Y_training.astype(np.int32)

lis_t = list(range(0,2520))
for t in range(2519):
    if t == 0:
        train = [(X_training[lis_t[t]],Y_training[lis_t[t+1]])]
    else:
        train.append((X_training[lis_t[t]],Y_training[lis_t[t+1]]))

C:\Users\bruce\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


In [8]:
##################################################################################
##################################################################################

class MyNetwork(Chain):

    def __init__(self, n_mid_units=100, n_out=60):
        super(MyNetwork, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            #self.l2 = L.Linear(n_mid_units, n_out)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        #return self.l2(h)
        h = F.relu(self.l2(h))
        return self.l3(h)
    
##################################################################################
##################################################################################



In [9]:
# Create an instance of the network we trained.
infer_model = MyNetwork()

In [11]:
# Load the parameter I sent
serializers.load_npz('beat_fume.model_0', infer_model)

In [13]:
x, t = train[count]

In [14]:
x

array([-0.7822677 ,  0.12208829, -1.7669486 , -0.20783566, -0.76660186,
       -1.1811434 ,  1.4242871 , -1.5588814 ,  0.6571092 , -0.581376  ],
      dtype=float32)

In [15]:
t

21

In [20]:
-0.015+30*0.0005

0.0

In [12]:
ans = []
ans_sig = []
pred = []
pred_sig = []

for count in range(50,100):

    # Get a test image and label
    x, t = train[count]
    print(t-15)
    ans.append(t-15)
    if t-15>0:
        ans_sig.append(1)
    else:
        ans_sig.append(0)
    
    print(x.shape, end=' -> ')
    x = x[None, ...]
    print(x.shape)
    
    # forward calculation of the model by sending X
    y = infer_model(x)
    
    # The result is given as Variable, then we can take a look at the contents by the attribute, .data.
    y = y.data
    
    # Look up the most probable digit number using argmax
    pred_label = y.argmax(axis=1)
    
    print('predicted label:', (pred_label[0]-15))
    pred.append(pred_label[0]-15)
    if pred_label[0]-15 >0:
        pred_sig.append(1)
    else:
        ans_sig.append(0)

13
(10,) -> (1, 10)
predicted label: 13
4
(10,) -> (1, 10)
predicted label: 4
19
(10,) -> (1, 10)
predicted label: 19
21
(10,) -> (1, 10)
predicted label: 21
12
(10,) -> (1, 10)
predicted label: 12
19
(10,) -> (1, 10)
predicted label: 19
12
(10,) -> (1, 10)
predicted label: 12
19
(10,) -> (1, 10)
predicted label: 19
19
(10,) -> (1, 10)
predicted label: 19
17
(10,) -> (1, 10)
predicted label: 17
17
(10,) -> (1, 10)
predicted label: 17
22
(10,) -> (1, 10)
predicted label: 22
12
(10,) -> (1, 10)
predicted label: 12
2
(10,) -> (1, 10)
predicted label: 2
4
(10,) -> (1, 10)
predicted label: 4
26
(10,) -> (1, 10)
predicted label: 26
15
(10,) -> (1, 10)
predicted label: 15
32
(10,) -> (1, 10)
predicted label: 32
17
(10,) -> (1, 10)
predicted label: 17
23
(10,) -> (1, 10)
predicted label: 23
14
(10,) -> (1, 10)
predicted label: 14
15
(10,) -> (1, 10)
predicted label: 15
18
(10,) -> (1, 10)
predicted label: 18
21
(10,) -> (1, 10)
predicted label: 21
16
(10,) -> (1, 10)
predicted label: 16
26
(10

In [23]:
subdata['returns'][1]*2000+30
train[0][1]

20

In [30]:
subdata['returns'][1]

-0.004830685328828133

In [29]:
(20-30)/2000

-0.005

-0.03504982412359723